In [1]:
# import numpy as np
# import cv2
# from keras.preprocessing import image
# import time

import numpy as np
import cv2
from keras.preprocessing import image
import time

import sys
import os
import glob
import re
import scipy
import tensorflow as tf
import align.detect_face
import time
import threading


# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
from keras.models import model_from_json


Using TensorFlow backend.
/home/topica/anaconda3/envs/workspace/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/topica/anaconda3/envs/workspace/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
###When you only want to dectect single face, uncomment this

# # facenet 
# image_size=160
# margin= 44
# gpu_memory_fraction=1.0

# def load_and_align_data(img, image_size,margin, gpu_memory_fraction):
#     minsize = 20 # minimum size of face
#     threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
#     factor = 0.709 # scale factor
#     with tf.Graph().as_default():
#         gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
#         sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
#         with sess.as_default():
#             pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
# #     img = scipy.misc.imread(os.path.expanduser(image_path))
#     img_size = np.asarray(img.shape)[0:2]
#     bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
#     if (len(bounding_boxes)==0):
#         bb=0
#         have_face = 0
#     else:
#         det = np.squeeze(bounding_boxes[0,0:4])
#         bb = np.zeros(4, dtype=np.int32)
#         bb[0] = np.maximum(det[0]-margin/2, 0)
#         bb[1] = np.maximum(det[1]-margin/2, 0)
#         bb[2] = np.minimum(det[2]+margin/2 - bb[0], img_size[1])
#         bb[3] = np.minimum(det[3]+margin/2 - bb[1], img_size[0])
#         have_face = 1
#     return bb,have_face

In [3]:
from keras.models import model_from_json
# model = model_from_json(open("/home/thaovu/tensorflow-101/model/facial_expression_model_structure.json", "r").read())
# model.load_weights('/home/thaovu/tensorflow-101/model/facial_expression_model_weights.h5') #load weights
model = model_from_json(open("/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.json", "r").read())
model.load_weights('/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.h5') #load weights

In [4]:
emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [5]:
###When you only want to dectect single face, uncomment this


# cap = cv2.VideoCapture("/home/topica/Downloads/2018-06-27_19h.wmv")
# # cap = cv2.VideoCapture("/home/topica/Video_topica/T6_08_20180816_090000.avi")
# frame = 0

# # fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# # out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))
# # out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (1024,700))

# while(True):
#     ret, img = cap.read()
#     detect_face, have_face= load_and_align_data(img,image_size,margin,gpu_memory_fraction)
#     if (have_face!=0):
#         detect_face = np.reshape(detect_face,(-1,4)) 
#         for (x,y,w,h) in detect_face:
#             detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
#             detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
#             detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48
#             cv2.rectangle(img,(x,y),(x+w,y+h),(64,64,64),2) #highlight detected face

#             img_pixels = image.img_to_array(detected_face)
#             img_pixels = np.expand_dims(img_pixels, axis = 0)

#             img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

#             #-----------------------------

#             predictions = model.predict(img_pixels) #store probabilities of 7 expressions
#             max_index = np.argmax(predictions[0])
#             print(max_index)

#             #background of expression list
#             overlay = img.copy()
#             opacity = 0.4
#             cv2.rectangle(img,(x+w+15,y-25),(x+w+150,y+115),(64,64,64),cv2.FILLED)
#             cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)

#             #connect face and expressions
#             cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255,255,255),1)
#             cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255,255,255),1)

#             emotion = ""
#             for i in range(len(predictions[0])):
#                 emotion = "%s %s%s" % (emotions[i], round(predictions[0][i]*100, 2), '%')

#                 """if i != max_index:
#                     color = (255,0,0)"""

#                 color = (255,0,0)

#                 cv2.putText(img, emotion, (int(x+w+15), int(y-12+i*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                
#     cv2.imshow('img',img)

#     frame = frame + 1
#     #print(frame)

#     #---------------------------------

#     if frame > 227:
#         break

#     if cv2.waitKey(70) & 0xFF == ord('q'): #press q to quit
#         break

# #kill open cv things
# # out.release()
# cap.release()
# cv2.destroyAllWindows()

In [6]:
### ###When you want to dectect multi face
image_size=160
margin= 44
gpu_memory_fraction=1.0
detect_multiple_faces = True

In [7]:
### ###When you want to dectect multi face
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
frame = 0
cap = cv2.VideoCapture(0)
count = np.zeros((7))

In [8]:
### ###When you want to dectect multi face
while(True):
    ret, img = cap.read()
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    start_time = time.time()
    bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    nrof_faces = bounding_boxes.shape[0]
    if nrof_faces>0:
        have_face = True
        det = bounding_boxes[:,0:4]
        det_arr = []
        result_face = []
        img_size = np.asarray(img.shape)[0:2]
        if nrof_faces>1:
            if detect_multiple_faces:
                for i in range(nrof_faces):
                    det_arr.append(np.squeeze(det[i]))
            else:
                bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                img_center = img_size / 2
                offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                det_arr.append(det[index,:])
        else:
            det_arr.append(np.squeeze(det))

        for i, det in enumerate(det_arr):
            det = np.squeeze(det)
            bb = np.zeros(4, dtype=np.int32)
            bb[0] = np.maximum(det[0]-margin/2, 0)
            bb[1] = np.maximum(det[1]-margin/2, 0)
            bb[2] = np.minimum(det[2]+margin/2, img_size[1])
            bb[3] = np.minimum(det[3]+margin/2, img_size[0])
            cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
#             scaled = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
            cv2.rectangle(img,(bb[0],bb[1]),(bb[2],bb[3]),(164,64,64),2) #highlight detected face
            detected_face = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY) #transform to gray scale
            detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48
#             print(detected_face.shape)
            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)

            img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

#             #-----------------------------
#             overlay = img.copy()
#             opacity = 0.4
#             cv2.rectangle(img,(bb[2]+10,bb[1]-25),(bb[2]+150,bb[1]+115),(64,64,64),cv2.FILLED)
#             cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)
#             #
#             #connect face and expressions
#             cv2.line(img,(int((bb[0]+bb[2])/2),bb[1]+15),(bb[2],bb[1]-20),(255,255,255),1)
#             cv2.line(img,(bb[2],bb[1]-20),(bb[2]+10,bb[1]-20),(255,255,255),1)
            
#             predictions = model.predict(img_pixels) #store probabilities of 7 expressions
#             max_index = np.argmax(predictions[0])
#             emotion = ""
#             for i in range(len(predictions[0])):
#                 emotion = "%s %s%s" % (emotions[i], round(predictions[0][i]*100, 2), '%')

#                 """if i != max_index:
#                     color = (255,0,0)"""

#                 color = (255,255,255)

#                 cv2.putText(img, emotion, (int(bb[2]), int(bb[1]-12+i*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            predictions = model.predict(img_pixels) #store probabilities of 7 expressions
            max_index = np.argmax(predictions[0])
            count[max_index] = count[max_index]+1
            result = "%s %s%s" % (emotions[max_index], round(predictions[0][max_index]*100, 2), '%')
            color = (55,55,255)
            cv2.putText(img, result, (int(bb[0]+5), int(bb[1]-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1) 
        print("--- %s seconds ---" % (time.time() - start_time))
    cv2.imshow('img',img)
    frame = frame + 1
    if cv2.waitKey(70) & 0xFF == ord('q'): #press q to quit
        break

#kill open cv things
# out.release()
cap.release()
cv2.destroyAllWindows()
print("the general emotion of the class: " + emotions[np.argsort(count)[5]])

--- 0.2817685604095459 seconds ---
--- 0.1661064624786377 seconds ---
--- 0.13174223899841309 seconds ---
--- 0.305922269821167 seconds ---
--- 0.23600196838378906 seconds ---
--- 0.15829682350158691 seconds ---
--- 0.15352201461791992 seconds ---
--- 0.2651398181915283 seconds ---
--- 0.3168778419494629 seconds ---
--- 0.22329354286193848 seconds ---
--- 0.3181192874908447 seconds ---
--- 0.12289905548095703 seconds ---
--- 0.1886615753173828 seconds ---
--- 0.2172868251800537 seconds ---
--- 0.33364105224609375 seconds ---
--- 0.13758397102355957 seconds ---
--- 0.13220524787902832 seconds ---
--- 0.11817550659179688 seconds ---
--- 0.20066523551940918 seconds ---
--- 0.13361287117004395 seconds ---
--- 0.1350255012512207 seconds ---
--- 0.13542771339416504 seconds ---
--- 0.283524751663208 seconds ---
--- 0.2818758487701416 seconds ---
--- 0.32808399200439453 seconds ---
--- 0.27704429626464844 seconds ---
--- 0.26369690895080566 seconds ---
--- 0.20087361335754395 seconds ---
--- 0

--- 0.30898427963256836 seconds ---
--- 0.17179298400878906 seconds ---
--- 0.3196842670440674 seconds ---
--- 0.33110952377319336 seconds ---
--- 0.3325023651123047 seconds ---
--- 0.28302979469299316 seconds ---
--- 0.3228116035461426 seconds ---
--- 0.16986727714538574 seconds ---
--- 0.30271029472351074 seconds ---
--- 0.2959113121032715 seconds ---
--- 0.3298466205596924 seconds ---
--- 0.25449538230895996 seconds ---
--- 0.36276745796203613 seconds ---
--- 0.35213661193847656 seconds ---
--- 0.30972886085510254 seconds ---
--- 0.2992517948150635 seconds ---
--- 0.16624665260314941 seconds ---
--- 0.2781362533569336 seconds ---
--- 0.2817270755767822 seconds ---
--- 0.3170292377471924 seconds ---
--- 0.14763903617858887 seconds ---
--- 0.14362168312072754 seconds ---
--- 0.15070557594299316 seconds ---
--- 0.3263993263244629 seconds ---
--- 0.16378235816955566 seconds ---
--- 0.2970578670501709 seconds ---
--- 0.22804856300354004 seconds ---
--- 0.17410588264465332 seconds ---
---

In [9]:
count

array([  8.,   0.,   7.,  99.,  48.,   4., 181.])